In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 3065, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 3065 (delta 9), reused 4 (delta 4), pack-reused 3054
Receiving objects: 100% (3065/3065), 411.06 MiB | 22.79 MiB/s, done.
Resolving deltas: 100% (2491/2491), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 19.4 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
from optuna.distributions import FloatDistribution, IntDistribution, CategoricalDistribution
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


# Configurations

In [5]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [6]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [7]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 0.2156


In [8]:
config = {
    'n_folds': 5,
    'build_folds': False,
    'train_ials': False,
    'train_ease_r': False,
    'train_nmf': False,
    'train_slim_bpr': False,
    'train_slim_elastic': False,
    'train_item_knn': True,
    'train_user_knn': True,
    'train_rp3': True,
    'train_p3a': True,
    'train_pure_svd': True,
    'train_pure_svd_scaled': True,
    'train_item_pure_svd': True,
    'save_github': True,
    'copy_ials_models': False
}

# Data Import

In [9]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [10]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [11]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [12]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender, ScaledPureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [13]:
for i in range(config['n_folds']):
    try:
        os.mkdir(f'/kaggle/working/fold_{i}')
    except FileExistsError:
        pass

In [14]:
# folds must be saved together with the models, otherwise the split might be wrong when tuning

folds = []

if config['build_folds']:
    splitter = CrossValidationSplitter(config['n_folds'])
    
    i = 0
    for URM_train, URM_valid in splitter.split(URM_all):
        folds.append((URM_train, URM_valid))
        sps.save_npz(f'/kaggle/working/fold_{i}/URM_train.npz', URM_train)
        sps.save_npz(f'/kaggle/working/fold_{i}/URM_valid.npz', URM_valid)
        
        if config['save_github']:
            upload_file(
                f'/kaggle/working/fold_{i}/URM_train.npz',
                f'models/{config["n_folds"]}Folds/fold_{i}/URM_train.npz', 
                f'CV {config["n_folds"]} folds: URM_train for fold {i}'
            )
            upload_file(
                f'/kaggle/working/fold_{i}/URM_valid.npz',
                f'models/{config["n_folds"]}Folds/fold_{i}/URM_valid.npz', 
                f'CV {config["n_folds"]} folds: URM_valid for fold {i}'
            )
        i += 1
        
else:
    for i in range(config['n_folds']):
        URM_train = sps.load_npz(f'/kaggle/working/Recsys_Challenge_2023/models/{config["n_folds"]}Folds/fold_{i}/URM_train.npz')
        URM_valid = sps.load_npz(f'/kaggle/working/Recsys_Challenge_2023/models/{config["n_folds"]}Folds/fold_{i}/URM_valid.npz')
        folds.append((URM_train, URM_valid))

# Training models with best params

In [15]:
if config['train_ials']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_ials_v2.json', 'r') as params_file:
        params = json.load(params_file)
        
    print('Training IALS models')
        
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = IALSRecommender(URM_train)
        recommender.fit(**params)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='ials'
        )
    
    # too big to be saved to github (need to be loaded from this notebook)

elif config['copy_ials_models']:
    
    for i in range(folds):
        shutil.copyfile(
            f'/kaggle/input/training-tuned-models-cv/fold_{i}/ials.zip', 
            f'/kaggle/working/fold_{i}/ials.zip', 
        )

In [16]:
if config['train_ease_r']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_EASE_R.json', 'r') as params_file:
        params_EASE_R = json.load(params_file)
        
    print('Training EASE_R models')
        
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = EASE_R_Recommender(URM_train)
        recommender.fit(**params_EASE_R)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='ease_r'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/ease_r.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/ease_r.zip', 
                f'CV {config["n_folds"]} folds: ease r for fold {i}'
            )

In [17]:
if config['train_nmf']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_NMF.json', 'r') as params_file:
        params_NMF = json.load(params_file)
        
    print('Training NMF models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = NMFRecommender(URM_train)
        recommender.fit(
            init_type="nndsvda", l1_ratio=params_NMF['l1_ratio'], beta_loss="frobenius",
            num_factors=params_NMF["num_factors"], solver=params_NMF["solver"]
        )

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='nmf'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/nmf.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/nmf.zip', 
                f'CV {config["n_folds"]} folds: nmf for fold {i}'
            )

In [18]:
if config['train_slim_elastic']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_slim.json', 'r') as params_file:
        params_slim_elastic = json.load(params_file)
        
    print('Training SlimElastic models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = SLIMElasticNetRecommender(URM_train)
        recommender.fit(**params_slim_elastic)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='slim_elastic'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/slim_elastic.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/slim_elastic.zip', 
                f'CV {config["n_folds"]} folds: slim elastic for fold {i}'
            )

In [19]:
if config['train_slim_bpr']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_Slim_BPR.json', 'r') as params_file:
        params_Slim_BPR = json.load(params_file)
        
    print('Training SlimBPR models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = SLIM_BPR_Python(URM_train)
        recommender.fit(**params_Slim_BPR)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='slim_bpr'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/slim_bpr.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/slim_bpr.zip', 
                f'CV {config["n_folds"]} folds: slim bpr for fold {i}'
            )

In [20]:
if config['train_item_knn']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
        params_item_knn = json.load(params_file)
    
    print('Training ItemKNN models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = ItemKNNCFRecommender(URM_train)
        recommender.fit(**params_item_knn)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='item_knn'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/item_knn.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/item_knn.zip', 
                f'CV {config["n_folds"]} folds: item_knn for fold {i}'
            )

Training ItemKNN models
ItemKNNCFRecommender: URM Detected 609 ( 4.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 241 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4493.41 column/sec. Elapsed time 4.97 sec
ItemKNNCFRecommender: Saving model in file '/kaggle/working/fold_0item_knn'
ItemKNNCFRecommender: Saving complete
ItemKNNCFRecommender: URM Detected 594 ( 4.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 237 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4629.30 column/sec. Elapsed time 4.83 sec
ItemKNNCFRecommender: Saving model in file '/kaggle/working/fold_1item_knn'
ItemKNNCFRecommender: Saving complete
ItemKNNCFRecommender: URM Detected 929 ( 7.1%) users with no interactions.
ItemKNNCFRecommender: URM Detected 540 ( 2.4%) items with no interactions.
Similarity column 22347 (100.0%), 4973.53 column/sec. Elapsed time 4.49 sec
ItemKNNCFRecommender: Saving model in file '/kaggle/working/fold_2item

In [21]:
if config['train_user_knn']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
        params_user_knn = json.load(params_file)
    
    print('Training UserKNN models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = UserKNNCFRecommender(URM_train)
        recommender.fit(**params_user_knn)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='user_knn'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/user_knn.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/user_knn.zip', 
                f'CV {config["n_folds"]} folds: user_knn for fold {i}'
            )

Training UserKNN models
UserKNNCFRecommender: URM Detected 609 ( 4.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 241 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4403.44 column/sec. Elapsed time 2.96 sec
UserKNNCFRecommender: Saving model in file '/kaggle/working/fold_0user_knn'
UserKNNCFRecommender: Saving complete
UserKNNCFRecommender: URM Detected 594 ( 4.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 237 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4770.45 column/sec. Elapsed time 2.73 sec
UserKNNCFRecommender: Saving model in file '/kaggle/working/fold_1user_knn'
UserKNNCFRecommender: Saving complete
UserKNNCFRecommender: URM Detected 929 ( 7.1%) users with no interactions.
UserKNNCFRecommender: URM Detected 540 ( 2.4%) items with no interactions.
Similarity column 13024 (100.0%), 5558.92 column/sec. Elapsed time 2.34 sec
UserKNNCFRecommender: Saving model in file '/kaggle/working/fold_2user

In [22]:
if config['train_rp3']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
        params_rp3 = json.load(params_file)
        
    print('Training RP3 models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = RP3betaRecommender(URM_train)
        recommender.fit(**params_rp3)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='rp3'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/rp3.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/rp3.zip', 
                f'CV {config["n_folds"]} folds: rp3 for fold {i}'
            )

Training RP3 models
RP3betaRecommender: URM Detected 609 ( 4.7%) users with no interactions.
RP3betaRecommender: URM Detected 241 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2816.57 column/sec. Elapsed time 7.93 sec
RP3betaRecommender: Saving model in file '/kaggle/working/fold_0rp3'
RP3betaRecommender: Saving complete
RP3betaRecommender: URM Detected 594 ( 4.6%) users with no interactions.
RP3betaRecommender: URM Detected 237 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2566.15 column/sec. Elapsed time 8.71 sec
RP3betaRecommender: Saving model in file '/kaggle/working/fold_1rp3'
RP3betaRecommender: Saving complete
RP3betaRecommender: URM Detected 929 ( 7.1%) users with no interactions.
RP3betaRecommender: URM Detected 540 ( 2.4%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2632.84 column/sec. Elapsed time 8.49 sec
RP3betaRecommender: Saving model in file '/k

In [23]:
if config['train_p3a']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
        params_p3alpha = json.load(params_file)
        
    print('Training P3alpha models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = P3alphaRecommender(URM_train)
        recommender.fit(**params_p3alpha)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='p3a'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/p3a.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/p3a.zip', 
                f'CV {config["n_folds"]} folds: p3a for fold {i}'
            )

Training P3alpha models
P3alphaRecommender: URM Detected 609 ( 4.7%) users with no interactions.
P3alphaRecommender: URM Detected 241 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2989.09 column/sec. Elapsed time 7.48 sec
P3alphaRecommender: Saving model in file '/kaggle/working/fold_0p3a'
P3alphaRecommender: Saving complete
P3alphaRecommender: URM Detected 594 ( 4.6%) users with no interactions.
P3alphaRecommender: URM Detected 237 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2988.55 column/sec. Elapsed time 7.48 sec
P3alphaRecommender: Saving model in file '/kaggle/working/fold_1p3a'
P3alphaRecommender: Saving complete
P3alphaRecommender: URM Detected 929 ( 7.1%) users with no interactions.
P3alphaRecommender: URM Detected 540 ( 2.4%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3334.02 column/sec. Elapsed time 6.70 sec
P3alphaRecommender: Saving model in file

In [24]:
if config['train_pure_svd']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
        params_pure_svd = json.load(params_file)
        
    print('Training PureSVD models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = PureSVDRecommender(URM_train)
        recommender.fit(num_factors=params_pure_svd['factors'])

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='pure_svd'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/pure_svd.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/pure_svd.zip', 
                f'CV {config["n_folds"]} folds: pure_svd for fold {i}'
            )

Training PureSVD models
PureSVDRecommender: URM Detected 609 ( 4.7%) users with no interactions.
PureSVDRecommender: URM Detected 241 ( 1.1%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.68 sec
PureSVDRecommender: Saving model in file '/kaggle/working/fold_0pure_svd'
PureSVDRecommender: Saving complete
PureSVDRecommender: URM Detected 594 ( 4.6%) users with no interactions.
PureSVDRecommender: URM Detected 237 ( 1.1%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.60 sec
PureSVDRecommender: Saving model in file '/kaggle/working/fold_1pure_svd'
PureSVDRecommender: Saving complete
PureSVDRecommender: URM Detected 929 ( 7.1%) users with no interactions.
PureSVDRecommender: URM Detected 540 ( 2.4%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing

In [25]:
if config['train_item_pure_svd']:
    # with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
    #     params_pure_svd_item = json.load(params_file)
    params_pure_svd_item = {"topK": 43, "num_factors": 171}
    
    print('Training PureSVDItem models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = PureSVDItemRecommender(URM_train)
        recommender.fit(**params_pure_svd_item)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='item_pure_svd'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/item_pure_svd.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/item_pure_svd.zip', 
                f'CV {config["n_folds"]} folds: item_pure_svd for fold {i}'
            )

Training PureSVDItem models
PureSVDItemRecommender: URM Detected 609 ( 4.7%) users with no interactions.
PureSVDItemRecommender: URM Detected 241 ( 1.1%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
PureSVDItemRecommender: Saving model in file '/kaggle/working/fold_0item_pure_svd'
PureSVDItemRecommender: Saving complete
PureSVDItemRecommender: URM Detected 594 ( 4.6%) users with no interactions.
PureSVDItemRecommender: URM Detected 237 ( 1.1%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
PureSVDItemRecommender: Saving model in file '/kaggle/working/fold_1item_pure_svd'
PureSVDItemRecommender: Saving complete
PureSVDItemRecommender: URM Detected 929 ( 7.1%) users with no interactions.
PureSVDItemRecommender: URM Detected 540 ( 2.4%) items with no interactions.
PureSVDItemRecommender: Computin

In [26]:
if config['train_pure_svd_scaled']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_scaled.json', 'r') as params_file:
        params_scaled = json.load(params_file)
        
    print('Training ScaledPureSVD models')
    
    for i, (URM_train, URM_valid) in enumerate(folds):
        recommender = ScaledPureSVDRecommender(URM_train)
        recommender.fit(**params_scaled)

        recommender.save_model(
            folder_path=f'/kaggle/working/fold_{i}', 
            file_name='pure_svd_scaled'
        )
        
        if config['save_github']: 
            upload_file(
                f'/kaggle/working/fold_{i}/pure_svd_scaled.zip',
                f'models/{config["n_folds"]}Folds/fold_{i}/pure_svd_scaled.zip', 
                f'CV {config["n_folds"]} folds: pure_svd_scaled for fold {i}'
            )

Training ScaledPureSVD models
ScaledPureSVDRecommender: URM Detected 609 ( 4.7%) users with no interactions.
ScaledPureSVDRecommender: URM Detected 241 ( 1.1%) items with no interactions.
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 6.12 sec
ScaledPureSVDRecommender: Saving model in file '/kaggle/working/fold_0pure_svd_scaled'
ScaledPureSVDRecommender: Saving complete
ScaledPureSVDRecommender: URM Detected 594 ( 4.6%) users with no interactions.
ScaledPureSVDRecommender: URM Detected 237 ( 1.1%) items with no interactions.
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 6.17 sec
ScaledPureSVDRecommender: Saving model in file '/kaggle/working/fold_1pure_svd_scaled'
ScaledPureSVDRecommender: Saving complete
ScaledPureSVDRecommender: URM Detected 929 ( 7.1%) users with no interactions.
ScaledPureSVDRecommender: URM Detected 540 ( 2.4%) item